In [1]:
# Tick speed threshold model
import numpy as np
import pandas as pd

import simple.histdb as db
from simple.ohlc import debounce
from simple.backtest import npBacktestMarket, getProfit, getLong, getShort
from simple.types import TTrade
from simple.pretty import pp
from simple.geneopt import GridOpt
from simple.plotly import interactTable, interactFigure
from simple.funcs import tickSpeed, getSpread
from simple.plotly import chartFigure

np.set_printoptions(edgeitems=2)
#%load_ext ipycache

In [ ]:
startDate, endDate = '2022-01-01', '2022-07-30'
ticker = 'BTCUSDT'
tickerID = db.getTickerID(f'{ticker}@BINANCE')
tickerID

KeyError('HISTDB_PASSWORD') .reconnectKeyError('HISTDB_PASSWORD') .reconnectKeyError('HISTDB_PASSWORD') .reconnectKeyError('HISTDB_PASSWORD') .reconnectKeyError('HISTDB_PASSWORD') .reconnectKeyError('HISTDB_PASSWORD') .reconnectKeyError('HISTDB_PASSWORD') .reconnect

In [ ]:
T = db.npTradeT(tickerID, startDate, endDate)
T.dtype = TTrade
T

In [ ]:
C = debounce(T)[:-1]
print(len(T), 'trades converted to', len(C), 'debounced records')
C

In [ ]:
#%%cache mycache.pkl ts, A, vA, B, vB
ts, A, vA, B, vB = db.flatSnapS(tickerID, startDate, endDate, 1)
ts

In [ ]:
R = getSpread(ts, A[0], B[0], C).view(np.recarray)
R

In [ ]:
np.savez_compressed(f'/tmp/{ticker}.npz', C=C, Ask=R.Ask, Bid=R.Bid)

In [ ]:
# declare chart linestyles
line_styles = {
    'Price': dict(color='gray', opacity=0.25),
    'Speed': dict(color='magenta', opacity=0.5, row=2),
    'Spread': dict(color='blue', opacity=0.5, row=3),

    'Profit': dict(color='blue', width=3, opacity=0.4, secondary_y=True, connectgaps=True),
    'RawPnL': dict(color='gray', width=3, opacity=0.4, secondary_y=True, connectgaps=True),
    'Buy': dict(mode='markers', color='green', symbol='triangle-up', size=10, line=dict(color="darkgreen", width=1)),
    'Sell': dict(mode='markers', color='red', symbol='triangle-down', size=10, line=dict(color="darkred", width=1))
}

In [ ]:
def model(Threshold: int = (2, 5.5, 0.125), Period: int = (500, 15000, 500)):
    Speed = tickSpeed(C, Period, log=True)
    D = npBacktestMarket(C.DT, R.Ask, R.Bid, Speed, Threshold, hold=None)
    P = getProfit(D)

    return P.Profit.sum(), {
        'Count': len(P),
        'AvgProfit': P.Profit.mean() if len(P) > 0 else 0,
        'RawPnL': P.RawPnL.sum() if len(P) > 0 else 0,
        'Fee': P.Fee.sum() if len(P) > 0 else 0,
        'MidPnL': P.MidPnL.sum() if len(P) > 0 else 0,
        'Sharpe': P.Profit.sum() / P.Profit.std() if len(P) > 1 else 0
     }

In [ ]:
def modelp(Threshold: int = (2, 5.5, 0.125), Period: int = (500, 15000, 500)):
    Price = C.Price
    Speed = np.nan_to_num(tickSpeed(C, Period, log=True))
    Spread = R.Mean

    D = npBacktestMarket(C.DT, R.Ask, R.Bid, Speed, Threshold, hold=600_000_000)
    P = getProfit(D)
    Profit = {'x': P.Index, 'y': P.Profit.cumsum()}
    RawPnL = {'x': P.Index, 'y': P.RawPnL.cumsum()}
    Fee = {'x': P.Index, 'y': P.Fee.cumsum()}
    MidPnL = {'x': P.Index, 'y': P.MidPnL.cumsum()}
    Buy, Sell = getLong(D), getShort(D)

    return P.Profit.sum(), locals()

In [ ]:
D = modelp(3.7, 500)
pd.DataFrame(D[1]['D'])

In [ ]:
interactFigure(modelp, height=600, rows=3, **line_styles)

In [ ]:
G = GridOpt(model)
G.fullSearch()
X = pd.DataFrame(G.log, columns=G.log_columns).drop_duplicates().sort_values('Fitness')
X.Threshold = X.Threshold.apply(lambda f: f'{f:1.2f}')  # explicit float format 
X

In [ ]:
# Grid map
pp(X.pivot(index='Period', columns='Threshold', values='MidPnL').astype(int))

In [ ]:
# Grid result browser
interactTable(modelp, X, height=600, rows=3, **line_styles)